In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
nr_classes = 10177 #unique identities
img_size = (218, 218) #taken as reference for conv. operations
flattened_kernel = 12 *12 * 64

In [0]:
#DEFINE MODEL

class Network(nn.Module):
  def __init__(self):
    super().__init__()
    #initialize plasticity parameters
    self.alpha = torch.nn.Parameter(torch.rand(12 * 12 * 64,
                                                nr_classes),
                                    requires_grad = True)
    self.hebb_trace = torch.nn.Parameter(torch.zeros(12 * 12 * 64,
                                                     nr_classes),
                                         requires_grad = False)
    self.eta = torch.nn.Parameter(1, requires_grad = True) #all conn. share same value of eta 

    #initialize "fixed" parameters, weights
    self.w = torch.nn.Parameter(torch.rand(12 * 12 * 64), nr_classes)

    #initialize layers
    self.cv1 = torch.nn.Conv2d(in_channels = 2, out_channels = 64
                                 kernel_size = 3, stride = 2)
    self.cv2 = torch.nn.Conv2d(64, 64, 3, stride = 2)
    self.cv2 = torch.nn.Conv2d(64, 64, 3, stride = 2)
    self.cv4 = torch.nn.Conv2d(64, 64, 3, stride = 2)
  
  def forward(self, x):
    #Convolutions,activation function (and pooling) to be discussed
    x = F.relu(self.cv1(x))
    x = F.relu(self.cv2(x_i))
    x = F.relu(self.cv3(x))
    x = F.relu(self.cv4(x))
    #Flatten
    x_i = x.view(-1, 12 * 12 * 64) #x_i.shape: [batch_size, flattened_kernel]
    #Plastic layer
    x_j = x_i.mm(self.w + torch.mul(self.alpha, self.hebb_trace)) #x_j.shape: [batch_size, nr_classes]
    x = F.softmax(x_j)
    hebb_trace = self.update_hebb_trace(
        self.eta, x_i, x_j, hebb_trace)
    return x, hebb_trace

  def update_hebb_trace(self, eta, x_i, x_j, hebb_trace):
    return self.eta * torch.mm(x_i.T, x_j) + hebb_trace * (1 - self.eta) #returns Tensor.shape: [flattened_kernel, nr_classes]
 
